### Assignment 4 on Tiny Imagenet dataset of 1 lakh training set belonging to 200 classes. Image dim - (64,64)

In [0]:
## Imports and initializations
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

import matplotlib.pyplot as plt
%matplotlib  inline

# Callbacks
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
#csv_logger = CSVLogger('resnet18_imagenet.csv')

batch_size = 128
nb_classes = 200
nb_epoch = 50
train_size = 100000
val_size = 10000


from keras.callbacks import ModelCheckpoint
dir = "/content/gdrive/My Drive/Colab Notebooks/EIP_2019/Assignment4/CustomWts/"
filepath = dir + "{epoch:03d}-{val_acc:.4f}.hdf5"
print(filepath)
!ls

# input image dimensions
img_rows, img_cols = 64, 64
# The imagenet images are RGB.
img_channels = 3

Using TensorFlow backend.


/content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/{epoch:03d}-{val_acc:.4f}.hdf5
sample_data


### Loading of imagenet data

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -qq '/content/gdrive/My Drive/Colab Notebooks/EIP_2019/Assignment4/tiny2-imagenet-200.zip'

val_data = pd.read_csv('./tiny-imagenet-200/val/val_annotations.txt', sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

train_datagen = ImageDataGenerator(rescale= 1./255,
                                  zoom_range = 0.25,
                                  shear_range = 0.1,                           
                                  width_shift_range=0.15,
                                  height_shift_range=0.15,
                                  rotation_range=55,
                                  horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory( r'./tiny-imagenet-200/train/', target_size=(img_rows, img_cols), color_mode='rgb', 
                                                    batch_size=batch_size, class_mode='categorical', shuffle=True, seed=42)
validation_generator = valid_datagen.flow_from_dataframe(val_data, directory='./tiny-imagenet-200/val/images/', x_col='File', y_col='Class', target_size=(img_rows, img_cols),
                                                    color_mode='rgb', class_mode='categorical', batch_size=batch_size, shuffle=True, seed=42)

print(len(train_generator), len(validation_generator))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
782 79


#### Number of conv blocks - 4, number of layers in each block - 4. 
#### There are 3 transition blocks. There are no dropout, dense layers.

In [0]:
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Flatten,
    Dense,
    Dropout
)
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers.merge import concatenate
from keras.regularizers import l2

def _bn_relu(input):  
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=3)(input)
    return Activation("relu")(norm)


def _conv_bn_relu(**conv_params):
    """Helper to build a conv -> BN -> relu block
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding, use_bias=False,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)

    return f

  
def _bn_relu_conv(**conv_params):
    """Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,
                      strides=strides, padding=padding, use_bias=False,
                      kernel_initializer=kernel_initializer,
                      kernel_regularizer=kernel_regularizer)(activation)

    return f

  
def create_block(input, num_filters, num_layers):
    temp = input
    for i in range(num_layers):
      input = _bn_relu_conv(filters=num_filters, kernel_size=(3, 3), strides=(1, 1))(input)
      print("shape inside block ", input.shape)
      num_filters *= 2    

    block = concatenate([temp, input])
    print("block shape after concat ", block.shape)
    return block
  
  
def create_transition(input):
    one_conv = _bn_relu_conv(filters=150, kernel_size=(1, 1), strides=(1, 1))(input)
    pool = MaxPooling2D(pool_size=(2, 2))(one_conv)
    print(one_conv.shape, "Inside transition", pool.shape)
    
    return pool
  
  
def output_layer(input):
    output = _bn_relu_conv(filters=nb_classes, kernel_size=(1, 1), strides=(1, 1))(input)
    output = MaxPooling2D(pool_size=(2, 2))(output)
    
    gap = GlobalAveragePooling2D()(output)  # in place of Convolution2D(10, 7)
    print(output.shape, " GAP ", gap.shape)
    output = Activation('softmax')(gap)
    print("Output ", output.shape)
    return output  
    
    
filters = 32
layers_in_blk = 4

input = Input(shape=(img_rows, img_cols, img_channels,))
print(type(input), " input shape ", input.shape)
conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3), strides=(1, 1))(input)
conv2 = _bn_relu_conv(filters=filters, kernel_size=(3, 3), strides=(1, 1))(conv1)
conv3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3), strides=(1, 1))(conv2)
print("conv3 shape ", conv3.shape)
        
block1 = create_block(conv3, filters, layers_in_blk)
trans1 = create_transition(block1)

block2 = create_block(trans1, filters, layers_in_blk)
trans2 = create_transition(block2)

block3 = create_block(trans2, filters, layers_in_blk)
trans3 = create_transition(block3)

block4 = create_block(trans3, filters, layers_in_blk)
output = output_layer(block4)



<class 'tensorflow.python.framework.ops.Tensor'>  input shape  (?, 64, 64, 3)
Instructions for updating:
Colocations handled automatically by placer.
conv3 shape  (?, 64, 64, 32)
shape inside block  (?, 64, 64, 32)
shape inside block  (?, 64, 64, 64)
shape inside block  (?, 64, 64, 128)
shape inside block  (?, 64, 64, 256)
block shape after concat  (?, 64, 64, 288)
(?, 64, 64, 150) Inside transition (?, 32, 32, 150)
shape inside block  (?, 32, 32, 32)
shape inside block  (?, 32, 32, 64)
shape inside block  (?, 32, 32, 128)
shape inside block  (?, 32, 32, 256)
block shape after concat  (?, 32, 32, 406)
(?, 32, 32, 150) Inside transition (?, 16, 16, 150)
shape inside block  (?, 16, 16, 32)
shape inside block  (?, 16, 16, 64)
shape inside block  (?, 16, 16, 128)
shape inside block  (?, 16, 16, 256)
block shape after concat  (?, 16, 16, 406)
(?, 16, 16, 150) Inside transition (?, 8, 8, 150)
shape inside block  (?, 8, 8, 32)
shape inside block  (?, 8, 8, 64)
shape inside block  (?, 8, 8, 12

#### Parameters are 1.96m instead of 23.9m of Resnet50. Best validation accuracy ~ 48.68% at epoch 73 and training accuracy~63.2%.

In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 3)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 32)   864         activation_1[0][0]               
__________________________________________________________________________________________________
batch_norm

In [0]:
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=2)
model.fit_generator(train_generator, steps_per_epoch=len(train_generator),                     
                    validation_data=validation_generator, validation_steps=len(validation_generator),
                    verbose=1,
                    initial_epoch = 0,
                    epochs=20,
                    max_queue_size=100,
                    callbacks=[checkpoint, lr_reducer, early_stopper])

Epoch 1/20
782/782 [==============================] - 899s 1s/step - loss: 5.1850 - acc: 0.0572 - val_loss: 5.6218 - val_acc: 0.0447
Epoch 2/20
782/782 [==============================] - 886s 1s/step - loss: 4.5335 - acc: 0.1179 - val_loss: 5.0299 - val_acc: 0.0887

Epoch 00002: val_acc improved from -inf to 0.08870, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/002-0.0887.hdf5
Epoch 3/20
782/782 [==============================] - 883s 1s/step - loss: 4.1260 - acc: 0.1706 - val_loss: 4.6641 - val_acc: 0.1178
Epoch 4/20
782/782 [==============================] - 887s 1s/step - loss: 3.8295 - acc: 0.2182 - val_loss: 4.7761 - val_acc: 0.1363

Epoch 00004: val_acc improved from 0.08870 to 0.13630, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/004-0.1363.hdf5
Epoch 5/20
782/782 [==============================] - 886s 1s/step - loss: 3.6053 - acc: 0.2568 - val_loss: 4.1038 - val_acc: 0.1939
Epoch 6/20
782/782 [==========

In [0]:
model.load_weights(dir + "018-0.3334.hdf5")
print("Loaded model from disk")
model.evaluate_generator(generator=validation_generator, steps=len(validation_generator), verbose=1)

Loaded model from disk
79/79 [==============================] - 31s 390ms/step


[3.4990049213409424, 0.3331]

In [0]:
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=2)
model.fit_generator(train_generator, steps_per_epoch=len(train_generator),                     
                    validation_data=validation_generator, validation_steps=len(validation_generator),
                    verbose=1,
                    initial_epoch = 20,
                    epochs=35,
                    max_queue_size=100,
                    callbacks=[checkpoint, lr_reducer, early_stopper])

Instructions for updating:
Use tf.cast instead.
Epoch 21/35
782/782 [==============================] - 881s 1s/step - loss: 2.7872 - acc: 0.4312 - val_loss: 3.2504 - val_acc: 0.3588
Epoch 22/35
782/782 [==============================] - 862s 1s/step - loss: 2.7562 - acc: 0.4366 - val_loss: 3.3678 - val_acc: 0.3436

Epoch 00022: val_acc improved from -inf to 0.34360, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/022-0.3436.hdf5
Epoch 23/35
782/782 [==============================] - 861s 1s/step - loss: 2.7337 - acc: 0.4420 - val_loss: 3.9522 - val_acc: 0.2849
Epoch 24/35
782/782 [==============================] - 861s 1s/step - loss: 2.7179 - acc: 0.4472 - val_loss: 3.2965 - val_acc: 0.3543

Epoch 00024: val_acc improved from 0.34360 to 0.35430, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/024-0.3543.hdf5
Epoch 25/35
782/782 [==============================] - 860s 1s/step - loss: 2.6974 - acc: 0.4491 - val_loss: 3.

In [0]:
print(k.eval(model.optimizer.lr))
model.load_weights(dir + "028-0.4367.hdf5")
print("Loaded model from disk")

model.evaluate_generator(generator=validation_generator, steps=len(validation_generator), verbose=1)

0.001
Loaded model from disk
79/79 [==============================] - 27s 339ms/step


[2.850449793624878, 0.4358]

In [0]:
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=2)
model.fit_generator(train_generator, steps_per_epoch=len(train_generator),                     
                    validation_data=validation_generator, validation_steps=len(validation_generator),
                    verbose=1,
                    initial_epoch = 29,
                    epochs=50,
                    max_queue_size=100,
                    callbacks=[checkpoint, lr_reducer, early_stopper])

Epoch 30/50
782/782 [==============================] - 856s 1s/step - loss: 2.6120 - acc: 0.4664 - val_loss: 3.5835 - val_acc: 0.3282
Epoch 31/50
782/782 [==============================] - 841s 1s/step - loss: 2.5869 - acc: 0.4725 - val_loss: 3.4580 - val_acc: 0.3365

Epoch 00031: val_acc improved from -inf to 0.33650, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/031-0.3365.hdf5
Epoch 32/50
782/782 [==============================] - 841s 1s/step - loss: 2.5818 - acc: 0.4748 - val_loss: 3.9711 - val_acc: 0.2891
Epoch 33/50
782/782 [==============================] - 842s 1s/step - loss: 2.5771 - acc: 0.4788 - val_loss: 3.2829 - val_acc: 0.3637

Epoch 00033: val_acc improved from 0.33650 to 0.36370, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/033-0.3637.hdf5
Epoch 34/50
782/782 [==============================] - 841s 1s/step - loss: 2.5667 - acc: 0.4801 - val_loss: 3.9036 - val_acc: 0.3029
Epoch 35/50
782/782 [====

In [0]:
model.fit_generator(train_generator, steps_per_epoch=len(train_generator),                     
                    validation_data=validation_generator, validation_steps=len(validation_generator),
                    verbose=1,
                    initial_epoch = 50,
                    epochs=60,
                    max_queue_size=100,
                    callbacks=[checkpoint, lr_reducer, early_stopper])

Epoch 51/60
782/782 [==============================] - 844s 1s/step - loss: 1.9602 - acc: 0.6035 - val_loss: 2.7584 - val_acc: 0.4627

Epoch 00051: val_acc improved from 0.43620 to 0.46270, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/051-0.4627.hdf5
Epoch 52/60
782/782 [==============================] - 843s 1s/step - loss: 1.9496 - acc: 0.6054 - val_loss: 2.8058 - val_acc: 0.4594
Epoch 53/60
782/782 [==============================] - 847s 1s/step - loss: 1.9420 - acc: 0.6077 - val_loss: 2.6440 - val_acc: 0.4850

Epoch 00053: val_acc improved from 0.46270 to 0.48500, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/053-0.4850.hdf5
Epoch 54/60
782/782 [==============================] - 839s 1s/step - loss: 1.9312 - acc: 0.6068 - val_loss: 2.7363 - val_acc: 0.4675
Epoch 55/60
782/782 [==============================] - 838s 1s/step - loss: 1.9244 - acc: 0.6085 - val_loss: 2.6785 - val_acc: 0.4797

Epoch 00055: val_acc 

In [0]:
model.fit_generator(train_generator, steps_per_epoch=len(train_generator),                     
                    validation_data=validation_generator, validation_steps=len(validation_generator),
                    verbose=1,
                    initial_epoch = 60,
                    epochs=70,
                    max_queue_size=100,
                    callbacks=[checkpoint, lr_reducer, early_stopper])

Epoch 61/70
782/782 [==============================] - 842s 1s/step - loss: 1.8375 - acc: 0.6277 - val_loss: 2.7443 - val_acc: 0.4678

Epoch 00061: val_acc did not improve from 0.48500
Epoch 62/70
782/782 [==============================] - 841s 1s/step - loss: 1.8401 - acc: 0.6267 - val_loss: 2.6826 - val_acc: 0.4781
Epoch 63/70
782/782 [==============================] - 837s 1s/step - loss: 1.8293 - acc: 0.6289 - val_loss: 2.6891 - val_acc: 0.4759

Epoch 00063: val_acc did not improve from 0.48500
Epoch 64/70
782/782 [==============================] - 840s 1s/step - loss: 1.8314 - acc: 0.6284 - val_loss: 2.6912 - val_acc: 0.4789
Epoch 65/70
782/782 [==============================] - 839s 1s/step - loss: 1.8277 - acc: 0.6287 - val_loss: 2.7453 - val_acc: 0.4713

Epoch 00065: val_acc did not improve from 0.48500
Epoch 66/70
782/782 [==============================] - 845s 1s/step - loss: 1.8203 - acc: 0.6298 - val_loss: 2.6783 - val_acc: 0.4774
Epoch 67/70
782/782 [======================

In [0]:
model.fit_generator(train_generator, steps_per_epoch=len(train_generator),                     
                    validation_data=validation_generator, validation_steps=len(validation_generator),
                    verbose=1,
                    initial_epoch = 70,
                    epochs=80,
                    max_queue_size=100,
                    callbacks=[checkpoint, lr_reducer, early_stopper])

Epoch 71/80
782/782 [==============================] - 851s 1s/step - loss: 1.8128 - acc: 0.6303 - val_loss: 2.6793 - val_acc: 0.4830

Epoch 00071: val_acc did not improve from 0.48500
Epoch 72/80
782/782 [==============================] - 846s 1s/step - loss: 1.7994 - acc: 0.6343 - val_loss: 2.6768 - val_acc: 0.4775
Epoch 73/80
782/782 [==============================] - 846s 1s/step - loss: 1.8075 - acc: 0.6324 - val_loss: 2.6192 - val_acc: 0.4868

Epoch 00073: val_acc improved from 0.48500 to 0.48680, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/073-0.4868.hdf5
Epoch 74/80
782/782 [==============================] - 847s 1s/step - loss: 1.8039 - acc: 0.6312 - val_loss: 2.6532 - val_acc: 0.4813
Epoch 75/80
782/782 [==============================] - 847s 1s/step - loss: 1.7955 - acc: 0.6326 - val_loss: 2.6444 - val_acc: 0.4819

Epoch 00075: val_acc did not improve from 0.48680
Epoch 76/80
 27/782 [>.............................] - ETA: 13:13 - loss: 

In [0]:
model.load_weights(dir + "073-0.4868.hdf5")
print("Loaded model from disk")
print(k.eval(model.optimizer.lr))

print(k.eval(model.optimizer.lr))
model.evaluate_generator(generator=validation_generator, steps=len(validation_generator), verbose=1)

Loaded model from disk
0.001
79/79 [==============================] - 26s 330ms/step


[2.6386054050445558, 0.4827]

In [0]:
from keras import optimizers
from keras.optimizers import Adam

model = Model(inputs=[input], outputs=[output])
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=.000031622778),
              metrics=['accuracy'])
model.load_weights(dir + "073-0.4868.hdf5")
print("Loaded model from disk")
print("Optimiser init with lr ", k.eval(model.optimizer.lr))

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', period=2)
model.fit_generator(train_generator, steps_per_epoch=len(train_generator),                     
                    validation_data=validation_generator, validation_steps=len(validation_generator),
                    verbose=1,
                    initial_epoch = 74,
                    epochs=84,
                    max_queue_size=100,
                    callbacks=[checkpoint, lr_reducer, early_stopper])

Loaded model from disk
Optimiser init with lr  3.1622778e-05
Epoch 75/84
782/782 [==============================] - 875s 1s/step - loss: 1.7899 - acc: 0.6364 - val_loss: 2.6738 - val_acc: 0.4780
Epoch 76/84
782/782 [==============================] - 860s 1s/step - loss: 1.7811 - acc: 0.6374 - val_loss: 2.6637 - val_acc: 0.4811

Epoch 00076: val_acc improved from -inf to 0.48110, saving model to /content/gdrive/My Drive/Colab Notebooks/Assignment4/CustomWts_2c/076-0.4811.hdf5
Epoch 77/84
782/782 [==============================] - 860s 1s/step - loss: 1.7718 - acc: 0.6411 - val_loss: 2.7121 - val_acc: 0.4732
Epoch 78/84
782/782 [==============================] - 860s 1s/step - loss: 1.7724 - acc: 0.6394 - val_loss: 2.7852 - val_acc: 0.4661

Epoch 00078: val_acc did not improve from 0.48110
Epoch 79/84
782/782 [==============================] - 860s 1s/step - loss: 1.7625 - acc: 0.6401 - val_loss: 2.6414 - val_acc: 0.4854
Epoch 80/84
782/782 [==============================] - 860s 1s/step

In [0]:
from keras.optimizers import Adam

model = Model(inputs=[input], outputs=[output])
model.load_weights(dir + "073-0.4868.hdf5")
print("Loaded model from disk")

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=.000010000001),
              metrics=['accuracy'])
print("Optimiser init with lr ", k.eval(model.optimizer.lr))
model.evaluate_generator(generator=validation_generator, steps=len(validation_generator), verbose=1)

Loaded model from disk
Optimiser init with lr  1.0000001e-05
79/79 [==============================] - 27s 342ms/step


[2.638605305099487, 0.4827]

At this point, **model is saturated** with training accuracy ~ 63.2% and validation accuracy ~ 48.68 at epoch 73. After this, train accuracy rises only slightly to 64.6%, but validation accuracy starts reducing.

Now, classes which have bad accuracy needs to be identified. And write a generator which trains these hard images from bad classes in more numbers.